In [2]:
import pymc as pm
import numpy as np
import arviz as az
import pandas as pd
import aesara.tensor as at
from aesara.tensor.subtensor import set_subtensor

%load_ext lab_black
%load_ext watermark

# Wine classification

Adapted from [Unit 10: italywines123.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit10/italywines123.odc).

Data can be found [here](https://archive.ics.uci.edu/ml/datasets/wine).

Associated lecture video: Unit 10 lesson 5

## Problem statement

This popular data set was provided by Forina et al (1988). 
The data below consist of results of a chemical analysis of wines grown in the same region in Italy but derived from three different cultivars. The analysis determined the quantities of 13 constituents found in each of the three types of wines. 

|Column|Variable|Description|
|---|---|---|
|1|Y|Type (Response, 1,2,3)|
|---|---|1 = [1 0 0]; 2 = [0 1 0]; 3 = [0 0 1]|
|2|X1|Alcohol|
|3|X2|Malic acid|
|4|X3|Ash|     
|5|X4|Alkalinity of ash| 
|6|X5|Magnesium|
|7|X6|Total phenols|
|8|X7|Flavanoids|
|9|X8|Nonflavanoid phenols||
|10|X9|Proanthocyanins|
|11|X10|Color intensity|
|12|X11|Hue|
|13|X12|OD280/OD315 of diluted wines|
|14|X13|Proline|

(a) Fit the multinomial regressionthat predicts the type of wine Y from predictors X1 - X13. What are estimated coefficients? What is the deviance?

(b) What is your prediction for pp=P(Ynew=1) if a new case has attributes ```new_attributes``` (below) How would you classify this wine type, as 1, 2, or 3?

Forina, M., Leardi, R., Armanino, C., and Lanteri, S. (1988). PARVUS: An extendable package of programs for data exploration, classification and correlation,  Elsevier, Amsterdam,   ISBN 0-444-43012-1;

Report at Institute of Pharmaceutical and Food Analysis and Technologies, Via Brigata Salerno, 16147 Genoa, Italy.

In [3]:
data = pd.read_csv("../data/wine.data", header=None)
Y = pd.get_dummies(data[0]).to_numpy()
X = data.drop(0, axis=1).to_numpy()
X_aug = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)

In [4]:
X_aug.shape, Y.shape

((178, 14), (178, 3))

In [5]:
with pm.Model() as m:
    X_data = pm.Data("X", X_aug, mutable=True)
    Y_data = pm.Data("y", Y, mutable=False)
    _b = pm.Normal("_b", 0, tau=0.05, shape=(14, 2))
    b = at.concatenate([at.zeros((14, 1)), _b], axis=1)
    
    # 178, 14 x 14, 3 -> 178, 3
    phi = pm.math.exp(pm.math.dot(X_data, b))
    # probabilities must sum to 1
    P = phi / pm.math.sum(phi, axis=1)[:, None]

    # P is 178, 3. category count determined by last axis
    pm.Multinomial("likelihood", n=1, p=P, observed=Y_data)

    trace = pm.sample(2000, init="jitter+adapt_diag_grad", target_accept=0.95)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag_grad...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [_b]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 81 seconds.


In [50]:
az.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
"_b[0, 0]",3.954,4.355,-4.033,12.404,0.048,0.039,8235.0,5664.0,1.0
"_b[0, 1]",-0.570,4.386,-8.882,7.394,0.039,0.053,12934.0,6074.0,1.0
"_b[1, 0]",3.104,1.427,0.594,5.865,0.031,0.023,2173.0,2692.0,1.0
"_b[1, 1]",0.276,1.866,-3.196,3.752,0.037,0.026,2545.0,3572.0,1.0
"_b[2, 0]",-3.669,1.646,-6.791,-0.650,0.030,0.022,3090.0,4199.0,1.0
"_b[2, 1]",0.953,2.191,-3.009,5.186,0.031,0.023,5111.0,5633.0,1.0
"_b[3, 0]",-7.786,3.033,-13.668,-2.251,0.052,0.036,3496.0,4802.0,1.0
"_b[3, 1]",0.597,4.049,-7.061,8.090,0.049,0.044,6713.0,5958.0,1.0
"_b[4, 0]",1.518,0.494,0.672,2.487,0.010,0.007,2619.0,4123.0,1.0
"_b[4, 1]",0.923,0.895,-0.738,2.596,0.015,0.011,3414.0,4365.0,1.0


Coefficients are not the same as BUGS results.

In [8]:
new_attributes = np.array(
    [1, 12.9, 2, 2.4, 17, 100, 2.8, 2.1, 0.35, 1.6, 5, 1.05, 3, 750]
).reshape((1, 14))
pm.set_data({"X": new_attributes}, model=m)
ppc = pm.sample_posterior_predictive(trace, model=m)

In [9]:
ppc.posterior_predictive.mean(
    dim=["chain", "draw", "likelihood_dim_0"]
).likelihood.values

array([0.91875 , 0.077375, 0.003875])

Getting the same predicted category, but with slightly different probabilities.

In [13]:
%watermark -n -u -v -iv -p aesara,aeppl

Last updated: Tue Jul 05 2022

Python implementation: CPython
Python version       : 3.10.4
IPython version      : 8.4.0

aesara: 2.7.3
aeppl : 0.0.31

pandas: 1.4.3
pymc  : 4.0.1
numpy : 1.23.0
aesara: 2.7.3
arviz : 0.12.1

